In [1]:
import os
import math
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema, find_peaks
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import cv2
import glob
import scipy.stats as stats

In [3]:
def local_maxima_points_plot_v2(current_veh_trj, final_local_maxima_points, de_final_local_maxima_points, current_change_point_record, min_speed, max_speed, current_veh_speed, do_denoise, de_cvs):
    
    id = current_change_point_record.veh_id
    state_index = math.floor(id/200)
    if state_index == 0:
        state_type = 'A-F'
    elif state_index == 1:
        state_type = 'A-D'
    elif state_index == 2:
        state_type = 'S-A'
    elif state_index == 3:
        state_type = 'F-D'
    else:
        state_type = 'D-S'
    start_rate = current_veh_trj['acceleration'].iloc[0]
    end_rate = current_veh_trj['acceleration'].iloc[-1]
    jerk = current_change_point_record['test_jerk']
        
    title_message = 'veh:'+str(id)+','+state_type+','+' start_rate:'+str(start_rate)+', end_rate:'+str(end_rate)+', jerk:'+str(jerk)
    plot_save_path = './figure_save_v2/' + 'veh_'+str(id)+'.png'
    
    plt.subplot(2, 1, 1)
    plt.title(title_message)
    plt.xlabel('Time Index')
    plt.ylabel('Speed')
    
    # bottom, top = plt.ylim()
    middle = 3
    marker_size = 5
    plt.plot(current_change_point_record.change_point_time_1, middle, color = '#ffacaa', marker='>', markersize=marker_size)
    plt.plot(current_change_point_record.change_point_time_2, middle, color = '#ffacaa', marker='<', markersize=marker_size)
    plt.plot(current_change_point_record.mid_change_point_time, middle, color = '#ffacaa', marker='*', markersize=marker_size)
    plt.axvline(current_change_point_record.change_point_time_1, color = '#ffacaa', marker='<', linestyle = ':', markersize=1, linewidth=1)
    plt.axvline(current_change_point_record.change_point_time_2, color = '#ffacaa', marker='>', linestyle = ':', markersize=1, linewidth=1)
    plt.axvline(current_change_point_record.mid_change_point_time, color = '#ffacaa', marker='*', linestyle = ':', markersize=1, linewidth=1)
    
    plt.plot(current_veh_trj.local_time, current_veh_trj.speed, label='Noise Data')
    if do_denoise == True:
        plt.plot(current_veh_trj.local_time, de_cvs, label='DeNoise Data',linestyle = '-.', linewidth=1)

    plt.ylim([min_speed, max_speed])
    plt.xlim([current_veh_trj.local_time.min(), current_veh_trj.local_time.max()])
    plt.grid()
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.xlabel('Time Index')
    plt.ylabel('Diff Scales')
    
    # bottom, top = plt.ylim()
    middle = 32
    marker_size = 5
    plt.plot(current_change_point_record.change_point_time_1 * 10, middle, color = '#ffacaa', marker='>', markersize=marker_size)
    plt.plot(current_change_point_record.change_point_time_2 * 10, middle, color = '#ffacaa', marker='<', markersize=marker_size)
    plt.plot(current_change_point_record.mid_change_point_time * 10, middle, color = '#ffacaa', marker='*', markersize=marker_size)
    plt.axvline(current_change_point_record.change_point_time_1 * 10, color = '#ffacaa', marker='>', label = 'start', linestyle = ':', markersize=1, linewidth=1)
    plt.axvline(current_change_point_record.change_point_time_2 * 10, color = '#ffacaa', marker='<', label = 'end', linestyle = ':', markersize=1, linewidth=1)
    plt.axvline(current_change_point_record.mid_change_point_time * 10, color = '#ffacaa', marker='*', label = 'changing point', linestyle = ':', markersize=1, linewidth=1)
    
    plt.plot(final_local_maxima_points.time_index*10, final_local_maxima_points.scale_id, 'o', markersize=1, label='local maxima points')
    plt.plot(de_final_local_maxima_points.time_index*10, de_final_local_maxima_points.scale_id, 'o', markersize=1, label='denoised points')
    
    plt.ylim([0, 64])
    plt.xlim([current_veh_trj.local_time.min()*10, current_veh_trj.local_time.max()*10])
    plt.grid()
    
    plt.legend()
    plt.savefig(plot_save_path, dpi=600)
    plt.close('all')